In [81]:
import pandas as pd
import numpy as np
import re
import os
from sklearn.cluster import KMeans
import ast
import itertools

In [82]:
#reading data from csv files
user_feat = pd.read_csv('user_features.csv')


user_feat['Calories'] = np.random.randint(150,500, size=len(user_feat))
user_feat['meal_size_rating'] = pd.DataFrame(np.random.randint(0,3, size = (len(user_feat),1)))
user_feat['previous_meal_size'] = pd.DataFrame(np.random.randint(0,3, size = (len(user_feat),1)))

print user_feat


spice_feat = pd.read_excel('test4.xls')
spice_feat = spice_feat.iloc[:,:7]
print spice_feat
#food user rating table
food_user = pd.DataFrame(np.random.randint(low=0, high=2, size=(len(spice_feat),len(user_feat))),columns=user_feat.user_guid)
food_user.index  =  spice_feat['food_id']
#print food_user.head()



    Age Sex          Location  NonVeg  user_guid  Calories  meal_size_rating  \
0    26   F   Jammu & Kashmir       1        500       458                 2   
1    33   F  Punjab & Haryana       0        501       364                 0   
2    93   M       South India       0        502       360                 1   
3    73   F       South India       1        503       223                 0   
4    95   F   Jammu & Kashmir       0        504       272                 2   
5    39   M  Punjab & Haryana       1        505       344                 1   
6    59   F         Rajasthan       1        506       399                 1   
7    55   M  Punjab & Haryana       1        507       416                 1   
8    99   F         Rajasthan       0        508       269                 1   
9    83   M  Punjab & Haryana       0        509       358                 2   
10   25   M         Rajasthan       1        510       345                 0   
11   58   M  Punjab & Haryana       0   

In [83]:
#Data Normalisation

def nor_age(age):
    
    if(age<=10):
        return 1
    elif(age>10 and age <=18):
        return 2
    elif(age>18 and age <=25):
        return 3
    elif(age>25 and age <=35):
        return 4
    elif(age>35 and age <=40):
        return 5
    elif(age>40 and age <=50):
        return 6
    elif(age>50 and age<=60):
        return 7
    else:
        return 8
    
    

s_dict = {"M":1,"F":5 }
l_dict = {'Jammu & Kashmir':2, 'Punjab & Haryana':5, 'Rajasthan':8, 'Maharashtra':10, 'South India':15}
user_feat['Age'] = user_feat['Age'].apply(nor_age)
user_feat  = user_feat.replace({"Sex":s_dict})
user_feat  = user_feat.replace({"Location":l_dict})

In [84]:
#dataset for content based recommender
#spice = np.array(spice_feat.iloc[:,0:4])
#print spice
#user_rating = pd.DataFrame(np.random.randint(low=-1, high=2, size=(len(spice_feat),len(user_feat))),columns=user_feat.user_guid)
#user_rating.index = spice_feat['food_id']
#user tells rating and spice tells if ingrediant is there or not.

In [85]:
#prepare dataset for 5 day window recommendation
import random
from random import randint

recommend_data = []
bought_data = []

for i in range(100):
    length = 3
    my_randoms = []
    for i in range(5):
        my_randoms.append(random.sample(xrange(0,len(spice_feat)), 3))
    recommend_data.append(my_randoms)
    
#print recommend_data

for i in range(100):
    length = randint(0,5)
    my_randoms = []
    for i in range(5):
        my_randoms.append(random.sample(xrange(0,len(spice_feat)), length))
    bought_data.append(my_randoms)

#print bought_data

recommends = []
for i in range(100):
    recommends.append(random.sample(xrange(0,len(spice_feat)), 3))

bought = []
for i in range(100):
    length = randint(0,5)
    bought.append(random.sample(xrange(0,len(spice_feat)), length))
    

new_user_feat = pd.DataFrame(user_feat.user_guid)
new_user_feat['last_5_days_recommend'] = recommend_data
new_user_feat['last_5_days_bought'] = bought_data
new_user_feat['recommends'] = recommends
new_user_feat['bought']  = bought
new_user_feat.to_csv('new_user_feat.csv', encoding='utf-8', index=False)
print new_user_feat.head

<bound method DataFrame.head of     user_guid                              last_5_days_recommend  \
0         500  [[33, 122, 85], [203, 60, 39], [110, 90, 19], ...   
1         501  [[140, 102, 215], [60, 26, 51], [120, 89, 27],...   
2         502  [[202, 106, 223], [205, 195, 224], [70, 0, 205...   
3         503  [[145, 5, 8], [142, 209, 84], [224, 39, 1], [1...   
4         504  [[183, 65, 212], [118, 127, 108], [49, 162, 20...   
5         505  [[215, 158, 16], [76, 185, 191], [130, 194, 15...   
6         506  [[42, 19, 112], [21, 43, 19], [195, 98, 105], ...   
7         507  [[135, 180, 189], [182, 13, 113], [121, 165, 2...   
8         508  [[186, 17, 156], [138, 87, 22], [178, 102, 51]...   
9         509  [[112, 203, 214], [30, 43, 106], [179, 140, 79...   
10        510  [[41, 91, 172], [12, 222, 147], [112, 46, 90],...   
11        511  [[153, 147, 14], [157, 214, 145], [172, 38, 10...   
12        512  [[46, 129, 220], [13, 126, 168], [139, 80, 123...   
13        513  [

In [86]:
def clustering_based(user_feat):    
    kmeans = KMeans(n_clusters = 5,random_state=0).fit(user_feat)
    user_feat['label'] = kmeans.labels_
    #print user_label
   
    user_feat = user_feat.sort_values(['label','Location'],ascending=[1,0])
    user = user_feat.drop_duplicates(subset=['Location','label'])
    user = user.groupby('label')['Location'].nlargest(3)
    return user,user_feat


In [87]:
def filter_food_loc(user_group,user_id,non_recom_list,user_feat,spice_feat,meal_type):
    index = np.where(user_feat['user_guid']==user_id)[0][0]
    print index
    #print user_feat['label']
    user_label = user_feat.loc[index,'label']
    loc = user_group[user_label].tolist()
    newspice_feat = spice_feat.set_index('food_id')
    spice = newspice_feat[newspice_feat['State'].isin(loc)]
    #print spice
    return filter_food_cal(spice,non_recom_list,user_feat,user_id,meal_type),loc 
    
def filter_food_cal(sspice,non_recom_list,user_feat,user_id,meal_type):
    index = np.where(user_feat['user_guid']==user_id)[0][0]
    sspice = sspice[sspice['Type']==user_feat.loc[index,'NonVeg']]
   
    #sspice = sspice[sspice['Meal Size']==next_meal_size]
    #sspice = sspice[sspice['Place in Menu']==meal_type]
    
    #print spice
    s_spice =  sspice.sort_values('count',ascending=[0])
    s_index = s_spice.index[~s_spice.index.isin(non_recom_list)]
    #print ' rec ',non_recom_list
    new_spices = s_spice[s_spice.index.isin(s_index)]
    #print new_spices
    new_spices = new_spices[new_spices['Calories']<=user_feat.loc[index,'Calories']]
    return new_spices

In [88]:
'''
def recommendation(user_group,user_id,non_recom_list,user_feat,spice_feat):
    new_spices,loc = filter_food_loc(user_group,user_id,non_recom_list,user_feat,spice_feat)
    recommend = []
    for i in range(min(3,len(new_spices))):
        recommend.append(new_spices.index[i])

    if(len(recommend)==0):
        other_state_spice = spice_feat[spice_feat['Location'].isin(loc)==False]

        other_state_spice = other_state_spice.sort('count',ascending= [0])
        new_other_state = other_state_spice[~other_state_spice.index.isin(non_recom_list)]

        for i in range(min(3,len(new_other_state))):
            recommend.append(new_other_state.index[i])
            
    return recommend    
'''



"\ndef recommendation(user_group,user_id,non_recom_list,user_feat,spice_feat):\n    new_spices,loc = filter_food_loc(user_group,user_id,non_recom_list,user_feat,spice_feat)\n    recommend = []\n    for i in range(min(3,len(new_spices))):\n        recommend.append(new_spices.index[i])\n\n    if(len(recommend)==0):\n        other_state_spice = spice_feat[spice_feat['Location'].isin(loc)==False]\n\n        other_state_spice = other_state_spice.sort('count',ascending= [0])\n        new_other_state = other_state_spice[~other_state_spice.index.isin(non_recom_list)]\n\n        for i in range(min(3,len(new_other_state))):\n            recommend.append(new_other_state.index[i])\n            \n    return recommend    \n"

In [89]:
def recommendation(user_group,user_id,non_recom_list,user_feat,spice_feat,meal_type):
    new_spices,loc = filter_food_loc(user_group,user_id,non_recom_list,user_feat,spice_feat,meal_type)
    
    recommend = []
    
    index = np.where(user_feat['user_guid']==user_id)[0][0]
    new_spices2 = spice_feat[spice_feat['Type']==user_feat.loc[index,'NonVeg']]   
    new_spices2 = new_spices2[new_spices2['Calories']<=user_feat.loc[index,'Calories']]
    print new_spices2
    
    #even in random,at least 2 constraints are there, calory and veg/non-veg and maybe location(to be confirmed) too
    x = randint(0,9)
    x=5
    if(x==5):
        #print ('random')  #to test
        index_to_append = new_spices2.index[randint(0,len(new_spices2))]
        #print random_feat.loc[index_to_append,'food_id']
        recommend.append(new_spices2.loc[index_to_append,'food_id'])
    
    
        for i in range(min(2,len(new_spices))):
            recommend.append(new_spices.index[i])
    
        if(len(recommend)<3):
            other_state_spice = spice_feat[spice_feat['State'].isin(loc)==False]
    
            other_state_spice = other_state_spice.sort_values('count',ascending= [0])
            new_other_state = other_state_spice[~other_state_spice.index.isin(non_recom_list)]

            if(len(new_other_state)>=(3-len(recommend))):
                for i in range(3-len(recommend)):
                    recommend.append(new_other_state.index[i])
                    
        if(len(recommend)<3):            
            newlymade_spice = spice_feat.sort_values('count',ascending = [0])            
            for i in range(3-len(recommend)):
                recommend.append(newlymade_spice.index[i])
            
    else:
        #print ('not random')
        for i in range(min(3,len(new_spices))):
            recommend.append(new_spices.index[i])
    
        if(len(recommend)<3):
            other_state_spice = spice_feat[spice_feat['State'].isin(loc)==False]
    
            other_state_spice = other_state_spice.sort_values('count',ascending= [0])
            new_other_state = other_state_spice[~other_state_spice.index.isin(non_recom_list)]
            if(len(new_other_state)>=(3-len(recommend))):
                for i in range(3-len(recommend)):
                    recommend.append(new_other_state.index[i])
        if(len(recommend)<3):            
            newlymade_spice = spice_feat.sort_values('count',ascending = [0])            
            for i in range(3-len(recommend)):
                recommend.append(newlymade_spice.index[i])
    
    return recommend

In [90]:
'''

# we have user_table which will give rating to a particular dish
# content based recommeder system
# we have 2 tables with us

#one is food - spices table and another is food-user rating table
#we will use tf-idf approach
#matrix of spice will contain only of 1 and 0's so need to normalize it

#normalization along the row
def spice_norm(spice):
    
    num_attr = np.sum(spice,axis=1)
    doc_vec = []
    for i in range(num_attr.shape[0]):
        if(num_attr[i] != 0): 
            doc_vec.append(np.sqrt(spice[i,:]*1.0/num_attr[i]))
        else:
            doc_vec.append(spice[i,:])
    return np.array(doc_vec)      
    
    
    
def attr_norm(spice):
    
    num_attr = np.sum(spice,axis=0)
    
    doc_vec = []
    for i in range(num_attr.shape[0]):
        if(num_attr[i] != 0):
            doc_vec.append(spice[:,i]*1.0 / num_attr[i])
        else:
            doc_vec.append(spice[:,i])
    return np.array(doc_vec)

def user_v(spice_mat,user):
    user_vec = []
    
    
    #print user[:,0]
    #print spice_mat[:,0]
    
    for i in range(user.shape[1]):
        user_vec.append([np.dot(user[:,i],spice_mat[:,j]) for j in range(spice_mat.shape[1])])
    
    print (user_vec)
    return np.array(user_vec)
    
def weight_predict(spice_mat,user):
        
    user_vec = user_v(spice_mat,user)
    
    preds = []
    #print spice_mat
    for i in range(spice_mat.shape[0]):
        preds.append([np.dot(spice_mat[i,:],user_vec[j,:]) for j in range(user_vec.shape[0])])

    return np.array(preds)
    
    
def idf_weight_pred(user,spice):
    spice_mat = spice_norm(spice)
    attr_wght = attr_norm(spice)
    #print attr_wght
    weighted_spice = []
    
    for i in range(spice_mat.shape[0]):
        weighted_spice.append([np.dot(spice_mat[i,:],attr_wght[:,j]) for j in range(attr_wght.shape[0])])
        
    preds  = weight_predict(np.array(weighted_spice),user)     
    
    return np.array(preds)
    




#print preds.shape


'''

"\n\n# we have user_table which will give rating to a particular dish\n# content based recommeder system\n# we have 2 tables with us\n\n#one is food - spices table and another is food-user rating table\n#we will use tf-idf approach\n#matrix of spice will contain only of 1 and 0's so need to normalize it\n\n#normalization along the row\ndef spice_norm(spice):\n    \n    num_attr = np.sum(spice,axis=1)\n    doc_vec = []\n    for i in range(num_attr.shape[0]):\n        if(num_attr[i] != 0): \n            doc_vec.append(np.sqrt(spice[i,:]*1.0/num_attr[i]))\n        else:\n            doc_vec.append(spice[i,:])\n    return np.array(doc_vec)      \n    \n    \n    \ndef attr_norm(spice):\n    \n    num_attr = np.sum(spice,axis=0)\n    \n    doc_vec = []\n    for i in range(num_attr.shape[0]):\n        if(num_attr[i] != 0):\n            doc_vec.append(spice[:,i]*1.0 / num_attr[i])\n        else:\n            doc_vec.append(spice[:,i])\n    return np.array(doc_vec)\n\ndef user_v(spice_mat,user

In [91]:
'''
def content_based_reco(user_rating,spice):
    preds = idf_weight_pred(np.array(user_rating),spice)
    outputdf = pd.DataFrame(data = preds,index = user_rating.index ,columns = user_rating.columns)
    return outputdf  

def recommendation_content(user_id,spice_feat,user_rating,non_recom_list,user_feat,outputdf):
    #spice_feat = np.array(spice_feat.iloc[:,0:4])
    #outputdf = content_based_reco(user_rating,spice_feat)
    
    spice = filter_food_cal(spice_feat,non_recom_list,user_feat,user_id)
    #print s_index
    spice.index = spice['food_id']
    output = outputdf[outputdf.index.isin(spice.index)]
    #print user_df.columns
    newoutput = output[user_id].sort_values(ascending = False)
    #print newoutput
    food_to_recommend = list(newoutput[0:3].index)

    return  food_to_recommend
    
'''

"\ndef content_based_reco(user_rating,spice):\n    preds = idf_weight_pred(np.array(user_rating),spice)\n    outputdf = pd.DataFrame(data = preds,index = user_rating.index ,columns = user_rating.columns)\n    return outputdf  \n\ndef recommendation_content(user_id,spice_feat,user_rating,non_recom_list,user_feat,outputdf):\n    #spice_feat = np.array(spice_feat.iloc[:,0:4])\n    #outputdf = content_based_reco(user_rating,spice_feat)\n    \n    spice = filter_food_cal(spice_feat,non_recom_list,user_feat,user_id)\n    #print s_index\n    spice.index = spice['food_id']\n    output = outputdf[outputdf.index.isin(spice.index)]\n    #print user_df.columns\n    newoutput = output[user_id].sort_values(ascending = False)\n    #print newoutput\n    food_to_recommend = list(newoutput[0:3].index)\n\n    return  food_to_recommend\n    \n"

In [92]:

def convert_list(row):
    #x = str(row['last_5_days_recommend'])
    y = str(row['last_5_days_bought'])
    #if isinstance(x,basestring):
    #    x = ast.literal_eval(x)
    if isinstance(y,basestring):
        y = ast.literal_eval(y)    
        
    #l1 =  list(itertools.chain.from_iterable(x))
    l2 =  list(itertools.chain.from_iterable(y)) 
    #print l1+l2
    return l2 

In [93]:
#for next day
#update the user_feat table for not recommenddation
def change_last_recommend(x,y):
    #print (row['last 5 days recommend'])
    new_recommend = []
    #print type(x) ,y,count 
    if isinstance(x,basestring):
        x = ast.literal_eval(x)
    if isinstance(y,basestring):
        y = ast.literal_eval(y)    
    for i in range(4):
        new_recommend.append(x[i+1])
    new_recommend.append(y)
    #print new_recommend
    return str(new_recommend)

In [94]:
global count
count=0

In [95]:
def recommend_items(user_id,non_recom_list,user,user_feat,spice_feat,meal_type):
    #print type(non_recom_list)
    l1 = recommendation(user,user_id,non_recom_list,user_feat,spice_feat,meal_type)
    
    #l2 = recommendation_content(user_id,spice_feat,user_rating,non_recom_list,user_feat,output_df)
    
    return str(l1)

In [96]:
def change_type(x):
    if isinstance(x,basestring):
        x = ast.literal_eval(x)
    return x    

In [97]:
def next_day_recommendation(user_feat,new_user_feat,spice_feat,meal_type):        #,user_rating):
    new_user_feat['last_5_days_recommend'] = new_user_feat.apply(lambda x: change_last_recommend(x['last_5_days_recommend'], x['recommends']), axis=1)
    new_user_feat['last_5_days_bought'] = new_user_feat.apply(lambda x: change_last_recommend(x['last_5_days_bought'], x['bought']), axis=1)
    new_user_feat['not_recommended'] = new_user_feat.apply(convert_list,axis=1)
    print new_user_feat['last_5_days_recommend'][0]
    print new_user_feat['last_5_days_bought'][0] 
    print new_user_feat['not_recommended'][0]
    user_group ,user_feat = clustering_based(user_feat)
    #spice = np.array(spice_feat.iloc[: ,0:4])
    #outputdf  = content_based_reco(user_rating, spice)
    new_user_feat['not_recommended'] = new_user_feat.apply(lambda x:change_type(x['not_recommended']),axis=1)
    #new_user_feat['not_recommended'].to_csv('not_recommended.csv',encoding='utf-8',index = False)
    #print new_user_feat
    new_user_feat['recommends'] = new_user_feat.apply(lambda x:recommend_items(x['user_guid'],x['not_recommended'],user_group,user_feat,spice_feat,meal_type),axis=1)
    print new_user_feat
    return new_user_feat
    
def main_recommendation(user_feat,spice_feat):      #,user_rating):
    new_user_feat = pd.read_csv('new_user_feat.csv')
    next_day = input('type 0 for recommending items to a user or 1 for shifting recommendations to the next day ') 
    if next_day == 1:
        meal_type = 0
        new_user_feat = next_day_recommendation(user_feat,new_user_feat,spice_feat,meal_type)
        new_user_feat.to_csv('new_user_feat.csv',encoding='utf-8',index = False)       
    else:
        user_id = input('enter the user_id')
        index = np.where(new_user_feat['user_guid']==user_id)[0]
        
        print new_user_feat.loc[index[0],'recommends']
        
        previous_rating = user_feat.loc[index[0],'meal_size_rating']
        previous_size = user_feat.loc[index[0],'previous_meal_size']
        next_meal_size = previous_size

        if(previous_rating==0 and previous_size<2):
            next_meal_size = previous_size + 1
        elif (previous_rating==2 and previous_size>0):
            next_meal_size = previous_size - 1
        
        
        #meal_type = input('enter the meal type(0,1,2,3)')
        print ("new meal size "+str(next_meal_size))

In [99]:
main_recommendation(user_feat,spice_feat)

type 0 for recommending items to a user or 1 for shifting recommendations to the next day 1
[[203, 60, 39], [110, 90, 19], [184, 78, 135], [138, 89, 36], [184, 189, 215]]
[[100, 17], [221, 128], [131, 148], [214, 222], [173, 34, 197, 135]]
[100, 17, 221, 128, 131, 148, 214, 222, 173, 34, 197, 135]
24
     food_id  State                        Dish  Type  Place in Menu  \
0          0      2                 Rogan Gosht     1              0   
1          1      2          Yoghurt Lamb Curry     1              0   
3          3      2                  Matschgand     1              0   
5          5      2                   Aab Gosht     1              0   
6          6      2                    Gushtaba     1              0   
13        13      2        Poo Cha (Butter Tea)     1              3   
32        32      5            Tandoori Chicken     1              0   
33        33      5              Butter Chicken     1              0   
34        34      5               Chicken Tikka   

90
     food_id  State                                Dish  Type  Place in Menu  \
2          2      2                            Dum Olav     0              0   
8          8      2                     Lyodur Tschaman     0              0   
9          9      2                          Herbal Tea     0              3   
10        10      2                                Skyu     0              2   
11        11      2                            Khameeri     0              1   
12        12      2                                Paba     0              2   
14        14      5                           Alu Matar     0              0   
15        15      5                  Moong Dal Parantha     0              1   
16        16      5                Paneer Chilli Cigars     0              2   
17        17      5     Hara Bhara Kebab in Green Gravy     0              2   
18        18      5                           Tava Naan     0              1   
19        19      5                  

61
     food_id  State                                Dish  Type  Place in Menu  \
2          2      2                            Dum Olav     0              0   
7          7      2                         Modur Pulav     0              1   
8          8      2                     Lyodur Tschaman     0              0   
9          9      2                          Herbal Tea     0              3   
10        10      2                                Skyu     0              2   
11        11      2                            Khameeri     0              1   
12        12      2                                Paba     0              2   
14        14      5                           Alu Matar     0              0   
15        15      5                  Moong Dal Parantha     0              1   
16        16      5                Paneer Chilli Cigars     0              2   
17        17      5     Hara Bhara Kebab in Green Gravy     0              2   
18        18      5                  

     food_id  State                        Dish  Type  Place in Menu  \
0          0      2                 Rogan Gosht     1              0   
3          3      2                  Matschgand     1              0   
5          5      2                   Aab Gosht     1              0   
6          6      2                    Gushtaba     1              0   
13        13      2        Poo Cha (Butter Tea)     1              3   
34        34      5               Chicken Tikka     1              2   
35        35      5                 Kunna Gosht     1              0   
37        37      5                        Paya     1              0   
38        38      5               Fried Chicken     1              0   
39        39      5  Bhatti Da Murga Pindiwalla     1              0   
40        40      5               Jeera Chicken     1              0   
41        41      5                Mutton Kadai     1              0   
42        42      5       Punjabi Chicken Lemon     1           

     food_id  State                                Dish  Type  Place in Menu  \
2          2      2                            Dum Olav     0              0   
7          7      2                         Modur Pulav     0              1   
8          8      2                     Lyodur Tschaman     0              0   
9          9      2                          Herbal Tea     0              3   
10        10      2                                Skyu     0              2   
11        11      2                            Khameeri     0              1   
12        12      2                                Paba     0              2   
14        14      5                           Alu Matar     0              0   
15        15      5                  Moong Dal Parantha     0              1   
16        16      5                Paneer Chilli Cigars     0              2   
17        17      5     Hara Bhara Kebab in Green Gravy     0              2   
18        18      5                     

     food_id  State                                Dish  Type  Place in Menu  \
2          2      2                            Dum Olav     0              0   
7          7      2                         Modur Pulav     0              1   
8          8      2                     Lyodur Tschaman     0              0   
9          9      2                          Herbal Tea     0              3   
10        10      2                                Skyu     0              2   
11        11      2                            Khameeri     0              1   
12        12      2                                Paba     0              2   
14        14      5                           Alu Matar     0              0   
15        15      5                  Moong Dal Parantha     0              1   
16        16      5                Paneer Chilli Cigars     0              2   
17        17      5     Hara Bhara Kebab in Green Gravy     0              2   
18        18      5                     

     food_id  State                                Dish  Type  Place in Menu  \
2          2      2                            Dum Olav     0              0   
7          7      2                         Modur Pulav     0              1   
8          8      2                     Lyodur Tschaman     0              0   
9          9      2                          Herbal Tea     0              3   
10        10      2                                Skyu     0              2   
11        11      2                            Khameeri     0              1   
12        12      2                                Paba     0              2   
14        14      5                           Alu Matar     0              0   
15        15      5                  Moong Dal Parantha     0              1   
16        16      5                Paneer Chilli Cigars     0              2   
17        17      5     Hara Bhara Kebab in Green Gravy     0              2   
18        18      5                     

IndexError: ('index 2 is out of bounds for axis 0 with size 2', u'occurred at index 79')